# Emerson Lab - Interview exercise

Below is simple way of processing a mechanism parts usage statistics.
1. Review the attached CSV files
    - SampleCSV.csv which is input data
    - ProductDescSample.csv which is augmentation data
2. Read files and create a complete data set
    - Any language you choose
    - Consideration that the Sample.csv may become much larger
3. Pick the most efficient format of file output and generate
4. Provide a PowerPoint that describes your solution and present


File Sample.csv

`SN` - The Part Serial number (Identified)

`Run` time - test/measurement run time

`Site` - Unknown

`Operation` - unknown

`Equipment` - unknown

`Status.1` - Overall the part measurement status: OK if all tests passed, NOK any test failed

`Name` - test/measurement name

`Input` - input value for the test

`Output` - output value for the current measurement

`High_Limit` - Allowed highest value for the output

`Low_Limit` - Allowed lowest value for the output

`Status.2` - current test/measurement status

The code below reads the CSV file at once


In [47]:
import pandas as pd

In [60]:
dtypes = {
    'SN': str,
    'Run_Time': str,
    'Site': str,
    'Operation': str,
    'Equipment': str,
    'Status': str,
    'Name': str,
    'Input': float,
    'Output': float,
    'High_Limit': float,
    'Low_Limit': float,
    'Status.1': str,
}
parse_date = ['Run_Time']

In [105]:
product_stats = pd.DataFrame(columns=['SN', 'Part_Desc', 'Run_Time', 'Site', 'Operation', 'Equipment', 'Status'])

df = pd.read_csv('data/dummySamples.csv', dtype=dtypes, parse_dates=parse_date, date_format='%m/%d/%y %H:%M:%s')
aug = pd.read_csv('data/dummyDescSample.csv')

In [106]:
df.rename(columns={'Status.1':'PartStatus'}, inplace=True)
df.rename(columns={'Status.2':'TestStatus'}, inplace=True)

In [107]:
aug_df = df.merge(aug, how='left', on='SN')

In [108]:
aug_df

,SN,Run_Time,Site,Operation,Equipment,PartStatus,Name,Input,Output,High_Limit,Low_Limit,TestStatus,Part_Desc
0,4085sv035,2025-04-11 01:43:36.653519,S4,0105,ST51,NOK,measure voltage,45.28,-0.61,0.0,-1.0,Pass,Hood_Cover
1,4085sv035,2025-04-11 01:43:36.653519,S4,0105,ST51,NOK,measure voltage,26.10,0.23,0.0,-1.0,Fail,Hood_Cover
2,4085sv035,2025-04-11 01:43:36.653519,S4,0105,ST51,NOK,measure voltage,54.25,0.62,0.0,-1.0,Fail,Hood_Cover
3,4085sv035,2025-04-11 01:43:36.653519,S4,0105,ST51,NOK,measure temperature,NaN,32.91,30.0,25.0,Fail,Hood_Cover
4,4085sv035,2025-04-11 01:43:36.653519,S4,0105,ST51,NOK,measure voltage,20.61,0.69,0.0,-1.0,Fail,Hood_Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61665,1028aw042,2025-04-16 01:43:33.653519,S3,0105,ST51,NOK,measure voltage,23.76,-0.83,0.0,-1.0,Pass,V8_Motor
61666,1028aw042,2025-04-16 01:43:33.653519,S3,0105,ST51,NOK,measure temperature,NaN,36.62,30.0,25.0,Fail,V8_Motor
61667,1028aw042,2025-04-16 01:43:33.653519,S3,0105,ST51,NOK,measure voltage,14.79,-0.43,0.0,-1.0,Pass,V8_Motor
61668,1028aw042,2025-04-16 01:43:33.653519,S3,0105,ST51,NOK,measure voltage,42.67,0.87,0.0,-1.0,Fail,V8_Motor


In [132]:
grouped_df = aug_df.groupby(['Part_Desc']).agg(
    failed_tests=('TestStatus', lambda x: (x != 'Pass').sum()),
    tests_overall=('TestStatus', 'count'),
    good_parts=('PartStatus', lambda x: (x == 'OK').sum()),
    bad_parts=('PartStatus', lambda x: (x == 'NOK').sum()),
    sn_tested=('SN', lambda x: x.nunique()),
)

In [133]:
grouped_df

,failed_tests,tests_overall,good_parts,bad_parts,sn_tested
Part_Desc,,,,,
Gear_Box,9808,15400,75,15325,3077
Hood_Cover,9917,15660,115,15545,3132
Panel,9805,15650,185,15465,3127
V8_Motor,9364,14960,115,14845,2992
